# Machine Learning

Algorithmen die sich durch das Nutzen von Daten automatisch optimieren

Welche Probleme können mit ML gelöst werden?

Kreditwürdigkeit, Preisvorhersagen, Spamfilter, ...

## Überwachtes Lernen

 - Man kennt Daten aus der Vergangenheit (Quadratmeter von Haus, Anzahl Zimmer, Garten vorhanden, ...)
 - Labeled: Man kennt den gewuenschten Output (Man kennt z.B. Preis zu dem Haus in Vergangenheit verkauft wurde. Man will neuen Verkaufspreis schaetzen)
 - Regression Task: Label das man vorhersagen will ist kontinuierlich (z.B. Euro-Preis)
 - Classification Task: Einordnen in eine bestimmte Kategorie (Handschrifterkennung)

## Unüberwachtes Lernen

 - Label nicht vorhanden
 - Label muss selbst gefunden werden

## Unterteilung der Daten
- X: Features (Daten die zur Schaetzung herangezogen werden)
- y: Label (Wert den wir vorhersagen wollen, z.B. Preis einer Taxifahrt)

### Training-Datensatz vs. Test-Datensatz
- Messen der Zuverlaessigkeit des trainierten Modells
- Trainieren mit Tainingsdatensatz und Verifizierung durch Testdatensatz

![title](images/train_test.png)

# Lineare Regression

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Linear_regression.svg/1024px-Linear_regression.svg.png" alt="MAE"
	title="Linear Regression" width="500" />

$\hat y = b_{0}x_{0} + ... + b_{n}x_{n}$

Ziel: Finden der $b_{n}$'s für bestee Gerade durch alle Punkte.

### Gradientenverfahren
<img src="https://blog.paperspace.com/content/images/2018/05/gd_basic.png" alt="MAE"
	title="Linear Regression" width="500" />

Source: https://blog.paperspace.com/content/images/2018/05/gd_basic.png

#  Scikit learn
- Beihnhaltet viele Machine Learning Algorithmen
- Algorithmen koennen schnell getauscht werden
- Algorithmen muessen nicht bis in die Tiefe verstanden werden
- Pragmatischer Ansatz

- X_train; y_train
- X_test; y_test

In [ ]:
from sklearn.model_selection import train_test_split ##Train-Test-Split erstellen

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y) ##Auch Prozentuale Angabe möglich

In [ ]:
model = from sklearn.model_family import Algorithm ##Importieren eines ML-Algorithmus der auf Daten angewendet wird

In [ ]:
model.fit(X_train, y_train) ##Das Modell wird trainiert

In [ ]:
predictions = model.predict(X_test) ##Vorhersagen für X_test treffen

In [ ]:
performance = error_metric(y_test, predictions) #Vergleich zwischen y_Test und Vorhersagen des Modells mit ausgewählter Error-Metrik

## Erstes kleines ML-Beispiel (ohne Parallelisierung)

Basiert auf folgendem Datensatz: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import sys

In [ ]:
n = 100  # CSV Einlesen und Datensatz verkleinern --> Nicht gut aber haben auf Jupyter-Node nicht viel Speicher da shared.
df = pd.read_csv('s3://nyc-tlc/trip data/green_tripdata_2019-02.csv', parse_dates=['lpep_pickup_datetime', 'lpep_dropoff_datetime'], header=0, skiprows=lambda i: i % n != 0)
## Beispiel: i=5 --> 5 mod 100 != 0 --> lambda gibt true zurück --> Zeile wird übersprungen

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df['pickup_hour'] = df['lpep_pickup_datetime'].dt.hour #Spalte mit 24 Kategorien für 24 Stunden hinzufügen

In [ ]:
df['pickup_hour'].tail(500)


In [ ]:
df['ride_duration'] = df['lpep_dropoff_datetime'].sub(df['lpep_pickup_datetime'], axis=0)

In [ ]:
df


In [ ]:
df['ride_duration_minutes'] = df['ride_duration'].dt.total_seconds().div(60).astype(int)

In [ ]:
df

In [ ]:
df = df[['passenger_count', 'trip_distance', 'fare_amount', 'total_amount', 'tip_amount','pickup_hour', 'ride_duration_minutes']] #Bereinigung für Pairplot

In [ ]:
fig = sns.pairplot(df)
fig.savefig("output.png")

In [ ]:
##Beispiel mit Matplotlib
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16,6)) ## 1 Zeile mit 3 Spalten
axes[0].plot(df['tip_amount'],df['total_amount'],'o')
axes[0].set_ylabel("Total Amount")
axes[0].set_title("Tip Amount")

axes[1].plot(df['fare_amount'],df['total_amount'],'o')
axes[1].set_ylabel("Total Amount")
axes[1].set_title("Fare Amount")

axes[2].plot(df['trip_distance'],df['total_amount'],'o')
axes[2].set_ylabel("Total Amount")
axes[2].set_title("Trip Distance")

plt.tight_layout()

In [ ]:
X = df[['trip_distance', 'pickup_hour', 'ride_duration_minutes']] ##Definieren der Labels
X

In [ ]:
y = df['tip_amount']
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.20, random_state=42) #Definieren der Train-Test-Aufteilung

In [ ]:
len(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
prediction = model.predict(X_test)
type(prediction)

In [ ]:
np.set_printoptions(threshold=sys.maxsize)
prediction

## Evaluation

Wir haben nun ein trainiertes Modell. Aber: Ist es gut??

<img src="https://i.imgur.com/19LNbyQ.jpg" alt="MAE"
	title="MAE" width="500" />
    
Source: https://stackoverflow.com/questions/56401346/mean-absolute-error-in-tensorflow-without-built-in-functions/56401550

Problem: Wenn nur einzelne Werte extrem abweichen würde das nicht erkannt werden......

Besser: Mean Squared Error (MSE): Durch das Quadrat werden einzelne starke Abweichungen stärker "bestraft"

<img src="https://cdn-media-1.freecodecamp.org/images/hmZydSW9YegiMVPWq2JBpOpai3CejzQpGkNG" alt="MAE"
	title="MAE" width="500" />

<img src="https://miro.medium.com/max/483/1*lqDsPkfXPGen32Uem1PTNg.png" alt="MAE"
	title="MAE" width="500" />


Root Mean Square Error: Bestraft einzelne starke Abweichungen stärker und zieht dann die Wurzel um die original Einheiten zu bekommen. Diese Variante wird oft verwendet.

## Frage: Was ist ein guter RMSE und was nicht?

Antwort: Es kommt drauf an.

Ein RMSE von 50€ wäre gut um z.B. den Preis einer Immobilie zu schätzen aber nicht für die Schätzung des Preises von Schokolade

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
df['tip_amount'].mean()

In [ ]:
np.sqrt(mean_squared_error(y_test, prediction))

Sehr schlechter Wert im Vergleich zum Durchschnittlichen Wert von tip_amount --> Kein gutes Modell 

## Cross Validation, Grid Search

- Viele ML Algorithmen sind mit Parametern anpassbar. Frage: Welche Parameter liefern die besten Ergebnisse?

- Idee: Ich stelle einen Parameter ein und trainiere das Modell und teste mit den Testdaten.

- Es ist nicht gut wenn z.B. stets die letzten 20% der Daten als Testdaten verwendet werden. Es kann sein, dass das trainierte Modell mit diesen Daten zufällig sehr gut oder sehr schlecht läuft.

- Kompletter Datensatz wird aufgeteilt in Training, Validation und Test. Der Test-Datensatz wird erst verwendet, wenn die Finalen Parameter gefunden wurden.

### k-fold Cross Validation
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/K-fold_cross_validation_EN.svg/1920px-K-fold_cross_validation_EN.svg.png" alt="MAE"
	title="MAE" width="500" />

1. Parameter des ML-Algos werden festgelegt.
2. Es werden k-Iterationen ausgeführt mit jeweils verschiedenen Test- und Trainingsdaten.
3. Error jeder Iteration wird berechnet.
4. Durschnitts-Error aller Iterationen bewertet die aktuelle Parameterkonfiguration des ML-Algos
5. Parameter verändern und Schritte 1 bis 4 wiederholen

In [ ]:
from sklearn.model_selection import cross_val_score
model = MLAlgo(paramter=xx)
score = cross_val_score(model, X_train, y_train, scoring='mean_squared_error', cv=5)

## Danach Durchschnitt der Fehler berechnen und dann für nächsten Parameter des ML Algos wiederholen

## Grid Search 

Bisher mussten Parameter des Algorithmus händisch angepasst werden. GridSearchCV automatisiert dies indem eine Liste an Parametern übergeben werden kann. Am Schluss lässt sich die Konfiguration anzeigen, die die beste Schätzungen geliefert hat.

In [ ]:
from sklearn import LogisticRegression
base_model = LogisticRegression()
param_grid = {'param1':[0.1, 0.2, ...], 'param2':[0.4,0.5,...] }

from sklearn.model_selection import GridSearchCV
grid_model = GridSearchCV(base_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=2)
grid_model.fit(X_train, y_train)

grid_model.best_estimator